# New topo DEMs in Southwest WA

In [ ]:
%matplotlib inline

In [ ]:
from pylab import *
from clawpack.geoclaw import topotools
from clawpack.visclaw import geoplot
from importlib import reload
import geopandas as gpd
import folium
import pandas as pd

In [ ]:
import find_topo_source

In [ ]:
eigth = 1/8  # half tile width
yrange = arange(47.0, 47.25+eigth, 0.25)
xrange = arange(-124.25, -123.5+eigth, 0.25)
name = 'NCEI tiles around Grays Harbor'
find_topo_source.make_combined_tile_kml(name, xrange, yrange)

In [ ]:
reload(find_topo_source)

In [ ]:
for x in xrange:
    for y in yrange:
        xm = x+eigth
        ym = y-eigth
        tile_name = find_topo_source.tile_coords(xm,ym,verbose=False)
        print(f'{x:.3f}, {y:.3f}:  {tile_name}')

### map of tiles

In [ ]:
gdf = gpd.read_file('NCEI_tiles_around_Grays_Harbor.kml', driver='KML')
m = folium.Map(location=(47,-124), zoom_start=9)
folium.GeoJson(gdf).add_to(m)

for x in xrange:
    for y in yrange:
        tile_name = find_topo_source.tile_coords(x,y,verbose=False)
        folium.Marker(
            location=[y-eigth,x+eigth],
            #popup = f"{x:.3f}, {y:.3f}: \n<b>Tile:</b>\n {tile_name}",
            popup = f"<b>Tile:</b>\n {tile_name}",
            tooltip="Click for info",
            icon=folium.Icon(color="red")  #, icon="cloud") # Customize the marker's appearance
        ).add_to(m) 
m

## Versions of this tile found on NCEI webpages:

In [ ]:
for x in xrange:
    for y in yrange:
        tile_name = find_topo_source.tile_coords(x,y,verbose=False)
        #print(f'{x:.3f}, {y:.3f}:  {tile_name}')
        print('-----------------------------------')
        tile_urls = find_topo_source.find_tile_url(tile_name)
        if len(tile_urls) == 0:
            print(f'{tile_name} not found')